In [152]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta, timezone
import time
import nvda_config
import twelvedata as td
from twelvedata import TDClient


In [153]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [154]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()




In [155]:

from datetime import datetime, timedelta
import pytz

# Define the start and end times in Eastern Time
eastern = pytz.timezone('US/Eastern')
start_time = eastern.localize(datetime.strptime("09:30", "%H:%M").replace(year=datetime.now().year, month=datetime.now().month, day=datetime.now().day))
end_time = eastern.localize(datetime.strptime("16:00", "%H:%M").replace(year=datetime.now().year, month=datetime.now().month, day=datetime.now().day))

# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000 

# Fetch data in chunks
current_date = (datetime.now(pytz.utc) - timedelta(minutes=1)).date()  # Corrected to get the date one minute ago in UTC
requests_made = 0
while True:
    current_time = datetime.now(pytz.utc).astimezone(eastern)
    
    if start_time.time() <= current_time.time() <= end_time.time():
       # Fetch time series data and indicators
        price_pdf = td.price(
            symbol=symbol
        )
        # Convert to pandas Dataframes
        price_json = price_pdf.as_json()
        price_df = pd.DataFrame(price_json)
        # Prepare for database insertion
        time_series_data = []

        # Process and align data
        for index, row in price_df.iterrows():
            # Extract the datetime string (already in correct format)
            datetime_str = row.name.strftime("%Y-%m-%d %H:%M:%S")

            time_series_data.append(
                (
                    symbol,
                    datetime_str,
                    row["price"]
                )
            )

        # Insert tick data into MySQL
        sql_time_series = """
        INSERT INTO nvda_new (symbol, datetime, price)
        VALUES (%s, %s, %s)
        """
        mycursor.executemany(sql_time_series, time_series_data)

        
    else:
        # Sleep for a short period before checking the time again
        time.sleep(10)
        print("markets closed")


markets closed
markets closed
markets closed


KeyboardInterrupt: 